# Larger and more frequent wildfires are cause for concern. 

Over the last few decades, wildfires have increased in both number and size across North America [(Schoennagel et al., 2017)](https://www.pnas.org/doi/abs/10.1073/pnas.1617464114). Regions such as the western United States have been hit particularly hard, where the problem is exacerbated by changes in climate, vegetation, and landuse. Not only are more swaths of land burning each year, but with urban sprawl forcing more people to build homes at and beyond the wildland-urban interface, increasing populations are put at risk of disaster. 

In the 20th century, fire supression policies because a standard practice of land management agencies in an attempt to control the ignition and growth of wildfires. Unfortuntely the enforcement of these practices sequested other traditional fire management practices of native and indigenous groups, leaving many of these groups more vulnerable. Through out native and indigenous histories, the practice of cultural burning - the intentional lighting of small, controlled fires to provide desired cultural and ecological services - was intrinsic to daily life and promoted a thriving ecosystem [(National Park Service, 2023)](https://nps.gov/subjects/fire/indigenous-fire-practices-shape-our-land.htm). Without cultural burns, build up of organic matter, debris, and/or tree falls in the landscape put these once managed landscapes and communities at risk of devasting fires. While these practices are slowly reestablishing their role in fire management, there is still a long way to go.

![Wildfire near U.S. 99 in Del Norte County, CA](https://gray-kptv-prod.cdn.arcpublishing.com/resizer/KFrafVIxIesbJCvVTE2Zcm5ujSU=/1200x675/smart/filters:quality(85)/cloudfront-us-east-1.images.arcpublishing.com/gray/DANRPXVYH5HNJOUNL53YXM2KPI.jpg)

Wildfire near U.S. 99 in Del Norte County, CA (credit: [Fox 12](https://www.kptv.com/2023/08/20/us-199-closed-near-oregon-border-evacuations-issued-california-wildfires/))

In [1]:
# Import standard packages
import os                           # Reproducible file names
import warnings                     # View warnings

# Import third part packages
import cartopy.crs as ccrs          # Cartographic projections
import earthpy as et                # File organization
import geopandas as gpd             # Enables work in geodataframes
import geoviews as gv               # Enables work with geographic data
import holoviews as hv              # For use with interactive plotting
import hvplot.pandas                # Plotting maps and plots
import pandas as pd                 # Work with dataframes
import pyogrio                       # Help with import of geodatabase

warnings.simplefilter(action='ignore')

### Native American Areas Dataset Description

The United States Geologic Survey (USGS) maintains a [National Boundary Dataset (NBD)](https://www.sciencebase.gov/catalog/item/5eaa545982cefae35a22231f) covering all 50 states and U.S. territories. The NBD includes boundaries including, but not limited to, political boundaries, national land holdings, U.S. Bureau holdings, military areas, and reserves managed by native communities. The Native American Area boundaries dataset within the NBD are derived from the [U.S. Census Bureau](https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html). These boundaries come from the Bureau's 2017 Boundary and Annexation Survey and is a combination of two datasets that include the American Indian/Alaska Native/ Native Hawaiian Areas national data and the Alaska Native Regional Corporation state-based data. Combined, these two datasets capture federally recognized American Indian reservations and off-reservation trust land areas, state-recognized American Indian reservations, Hawaiian homelands, and 12 Alaska Native Regional Corporations.

**Data Citation:**
U.S. Geological Survey, National Geospatial Technical Operations Center, 2023, National Boundary Dataset (NBD), Native American Areas - [USGS National Map Downloadable Data Collection](https://www.sciencebase.gov/catalog/item/4f70b219e4b058caae3f8e19): U.S. Geological Survey, Reston, Virginia. Published 08/28/2023. Accessed 10/18/2023.

In [2]:
# Download national boundary data from the NBD Website
# Metadata XML File (https://thor-f5.er.usgs.gov/ngtoc/metadata/waf"
# "/boundaries/nbd/filegdb101/GovernmentUnits_National_GDB.xml")
# Data is a geodatabase of national boundaries from the entire nation
# Note, to download large data, may need to change machine type of codespace
nbd_url = (
    "https://prd-tnm.s3.amazonaws.com/StagedProducts/GovtUnit/National/GDB"
    "/GovernmentUnits_National_GDB.zip")

nbd_dir = et.data.get_data(url=nbd_url)    # Path to downloaded file directory
# nbd_dir

In [3]:
# Import the Native American Areas (NAA) layer
# Call the GDB file 
# earth-analytics/data/earthpy-downloads/GovernmentUnits_National_GDB/GovernmentUnits_National_GDB.gdb
nbd_path = os.path.join(nbd_dir, 'GovernmentUnits_National_GDB.gdb')

# Read in the GDB and layer of interest (Native American Areas)
nbd_naa_gdf = gpd.read_file(nbd_path, 
                            layer='GU_NativeAmericanArea', from_disk=True)
# nbd_naa_gdf.head()

In [4]:
# Test plot to demonstrate successful import of data

# nbd_naa_gdf.plot()

### Wildfire Dataset Description

The wildfire data used in this analysis comes from the [USDA's wildfire occurance data for the United States, 1992-2020](https://www.fs.usda.gov/rds/archive/Catalog/RDS-2013-0009.6). This is the fifth version of the dataset and was generated from reporting systems of federal, state, and local fire organizations. Wildfire occurences are reported from 1992-2020 and include 2.3 million wildfire records. 

**Data Citation:**
Short, Karen C. 2022. Spatial wildfire occurrence data for the United States, 1992-2020 [FPA_FOD_20221014]. 6th Edition. Fort Collins, CO: Forest Service Research Data Archive. https://doi.org/10.2737/RDS-2013-0009.6

In [5]:
# Download wildfire occurance data from geodatabase

fire_url = (
    "https://www.fs.usda.gov/rds/archive/products/RDS-2013-0009.6"
    "/RDS-2013-0009.6_Data_Format2_GDB.zip"
)

fire_dir = et.data.get_data(url=fire_url)   # Path to downloaded fire directory
# fire_dir

In [6]:
# Import fires layer from geodatabase (cashing data)
# Put import in an if statement to check if data already downloaded
fire_path = os.path.join(fire_dir, 'Data','FPA_FOD_20221014.gdb')
if not 'fire_gdf' in globals():
    # print('fire_gdf does not exist. Loading...')
    fire_gdf = pyogrio.read_dataframe(fire_path, layer='Fires')

# fire_gdf.head()                     # Prints only first few lines of data

In [7]:
# Clean up the data (Lat/Long already assigned to geometry)
# Create an updated geodatabase, populating specific variables
# [[]] Two sets, one to search and one to define a list
fire_clean_gdf = (
    fire_gdf
    [['FOD_ID', 'DISCOVERY_DATE', 'FIRE_SIZE', 'geometry']]
    .set_index('FOD_ID')
)

# Convert the existing date to a datetime format
fire_clean_gdf.DISCOVERY_DATE = pd.to_datetime(fire_clean_gdf.DISCOVERY_DATE)

# Reproject dataframe to match the CRS of NBD boundary
# print('Geodetic CRS before reprojection: ' + str(fire_clean_gdf.crs))
fire_clean_gdf = fire_clean_gdf.to_crs(nbd_naa_gdf.crs)
# print('Geodetic CRS after reprojection: ' + str(fire_clean_gdf.crs))

# fire_clean_gdf                  # Use .info() to see data types

In [8]:
# Spatially join the NAA with the fire history geodataframes
fire_region_gdf = (
    nbd_naa_gdf[['GNIS_NAME', 'geometry']]
    .sjoin(fire_clean_gdf, how='inner', predicate='intersects')
)

# Calculate max fire size for each year in NAA combination
fire_region_gdf = (fire_region_gdf
    .groupby(['GNIS_NAME', fire_region_gdf.DISCOVERY_DATE.dt.year])
    .agg(
        max_fire_size=('FIRE_SIZE','max'),      # New name = old, how
        num_fires=('index_right', 'count'))     # New name = old, how
)

# print('Total number of fires: ' + str(fire_region_gdf.num_fires.sum()))
# fire_region_gdf

In [9]:
# Calculate the area of each NAA (use Albers Equal Area Projection)
nbd_naa_gdf['area_ha'] = (        # Add an area column to gdf
    nbd_naa_gdf.to_crs(9822)      # Convert new temp gdf to Albers epsg
    .area/10000/1000000           # Calculate the NAA area (millions ha)
)                                 # Note 1 hectare is 10,000 sq.m.

# Calculate total number of fires in each NAA region
fire_count_df = (fire_region_gdf  # Note that this is a regualar dataframe
    .reset_index()                # Reset because name is currently in index
    [['GNIS_NAME', 'num_fires']]
    .groupby('GNIS_NAME')         # Group by the name of the region
    .sum()                        # Sum of the number of fires 
)
fire_density_gdf=(nbd_naa_gdf     # Note that this will be a geodataframe
    .set_index('GNIS_NAME')
    .join(fire_count_df)          # Join nbd to fire_density gdf
    [['num_fires', 'area_ha', 'geometry']] # Add area and geomety back in
)

# Calculate fire density (number of fires per area)
fire_density_gdf['fire_density_per_ha'] = (
    fire_density_gdf.num_fires / fire_density_gdf.area_ha
)

# Print the index
# fire_density_gdf.fire_density_per_ha

### These plots show both the largest fires on record and the total number of fires per year in each of the officially recognized Native American and Indigenous Region within the United States and its territories. Regions can be interactively selected to toggle between regions. 

In [10]:
# Set values for ylabels and titles
labels = pd.DataFrame(dict(
    column_name = ['max_fire_size', 'num_fires'],
    ylabel = ['Fire Size (million ha)', 'Number of Fires'],
    title = ['Largest fire on record in the region', 
             'Total number of fires in the region']))

def fire_plot(region_name, df=fire_region_gdf, labels=labels):
    """
    Create a multi-panel plot for a region

    Parameters
    ----------
    region_name : str
      The name of the region to generate a plot for. Must exists 
      in the 'name' index of df.
    df : pd.DataFrame
      The dataframe with the data to plot. Columns much match
      an item in labels.column_name to be plotted
    labels : pd.DataFrame
      Plot labels. Must have a 'column_name', 'ylabel', and 'title'
      columns with str values. Each row will be a subplot.

    Returns
    -------
    plot : hv.core.layout.Layout
      A holoviews plot layout or similar. For use with hv.DynamicMap.
    """
    # Generate a subplot for each row in the labels
    subplots = []
    # Iterate through the labels row by row (labels is a gdf)
    for i, labs in labels.iterrows():
        # Create subplot (for each region name row in gdf, create an hvplot)
        subplot = (
            df.xs(region_name, level='GNIS_NAME')  # Look for name in name index
            [[labs.column_name]]              # Take column name in labels gdf
            .hvplot(
                xlabel='Year', ylabel=labs.ylabel,
                title=labs.title,
                width=1000,                   # Set plot width
                color='red', size=2           # Set line color and width
            ))
        subplots.append(subplot)              # Accumulate subplots in a list

    # Stack the subplots vertically
    plot = hv.Layout(subplots).cols(1)        # Use holoview, combine subplots
    return plot

# Create a dropdown menu to switch between regions
(
    hv.DynamicMap(
        # The plotting function for the two-panel fire history
        fire_plot,
        # Define the dimension for the dropdown
        kdims=[('region', 'Region')])         # (name of index, label name)
    # Add the explicit indexing - region names as a bokeh dimension
    .redim.values(region=fire_region_gdf.reset_index().GNIS_NAME)
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'aa16f619-a99f-40d9-a3c0-f16d35d325fc': {'version…

In [11]:
# Divide out Alaska and Continental Areas
# Alaska = 'Alaska Native Regional Corporation - 642'
# Continental US = 'Native American Reservation - 640'

# fire_density_ftype = fire_density_gdf.merge(nbd_naa_gdf, on='GNIS_NAME')
# fire_density_ftype.head()

# ftype_val = fire_density_ftype['FTYPE'].tolist()
# ftype_val
# print(pd.unique(ftype_val['FTYPE']))

# alaska_df = fire_density_ftype[fire_density_ftype["FTYPE"] == "642"]
# alaska_df.head()

# continental_df = fire_density_gdf[fire_density_ftype["FTYPE"] != "642"]
# continental_df.head()

# Notes:
# df is the fire df
# FType is the attribute in the table
# alaska_df = df[df["FTYPE"]  == "Alaska Rez"]
# continental_df = df[df["FTYPE"] != "Alaska Rez"]

# To plot as two side by side plots, need to find the max and min of the fire 
# density column being plotted. Use this to plot the two scales the same 
# (clim = (min_num, max_num)) ... us this in the (.opts)
# This esentially sets the min and max colorbar range to these levels


In [12]:
# Simplify the gdf geometries
fire_density_gdf.geometry = (
    fire_density_gdf.geometry.simplify(tolerance=0.1)
)

In [13]:
# Histogram of the data to see potential skew

# fire_density_gdf.plot('fire_density_per_ha', kind="hist")


### The chloropleth map shows the total number of fires for all of the recognized Native American and Indigenous Areas in the United States and its territories.

In [14]:
# Create a chloropleth map of numer of fires (geoviews polygons geometry)
poly_plot = (
    gv.Polygons(fire_density_gdf            # Use Geoviews polygon geometry
                .drop(                      # Remove Alaska Aleutian Area
                    ['Aleut Alaska Native Regional Corporation'],
                      axis='rows')
                .reset_index()              # Reset index to extract 'NAME'
                .dropna()                   # Remove NAN values
                [['num_fires', 'geometry']])
    .opts(                                  # Set plotting options
        width=1000, height=600,
        # cnorm (default='linear'): str Color scaling which must be one 
        # of 'linear', 'log' or 'eq_hist'
        cnorm='linear',
        colorbar=True, color='fire_density',
        cmap='plasma', line_color='white',
        xaxis='bare', yaxis='bare', tools=['hover']
    )
)
((gv.tile_sources.OSM * poly_plot).opts(
    data_aspect=1,
    # projection=ccrs.PlateCarree(central_longitude=-121))
))

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (num_fires)

### The chloropleth map shows the fire density (number of fires per million hectares) on a log scale for all of the recognized Native American and Indigenous Areas in the United States and its territories.

In [15]:
# Create a chloropleth map of fire density (geoviews polygons geometry)
poly_plot = (
    gv.Polygons(fire_density_gdf            # Use Geoviews polygon geometry
                .drop(                      # Remove Alaska Aleutian Area
                    ['Aleut Alaska Native Regional Corporation'],
                      axis='rows')
                .reset_index()              # Reset index to extract 'NAME'
                .dropna()                   # Remove NAN values
                [['fire_density_per_ha', 'geometry']])
    .opts(                                  # Set plotting options
        width=1000, height=600,
        # cnorm (default='linear'): str Color scaling which must be one 
        # of 'linear', 'log' or 'eq_hist'
        cnorm='log',
        colorbar=True, color='fire_density',
        cmap='plasma', line_color='white',
        xaxis='bare', yaxis='bare', tools=['hover']
    )
)
((gv.tile_sources.OSM * poly_plot).opts(
    data_aspect=1,
    # projection=ccrs.PlateCarree(central_longitude=-121))
))

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (fire_density_per_ha)

In [16]:
%%capture
%%bash
jupyter nbconvert nbd_fire_naa.ipynb --to html --no-input